# Import libraries

In [82]:
from text_matcher.matcher import Text, Matcher
import json
import pandas as pd
import random
from IPython.display import clear_output
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 6]
#pd.set_option('display.max_colwidth', None)

# Load in our Data Files

🛑 Input a link to the json file of articles to run the spell-check on


In [83]:
# Load in the JSON file with our JSTOR articles and data from TextMatcher
# (Note: must have the file 'default.json' in the same directory as this notebook)
articles = pd.read_json('../algorithm-testing/jstor-gender-trouble-all-articles.jsonl', lines=True)

# Importing the spell-check package

Languages tested:
* English - ‘en’
* Spanish - ‘es’
* French - ‘fr’
* German - ‘de’

Make sure to run `pip install pyspellchecker` in terminal before running the cell below

Recommended upper bound of anomaly beyond 75% percentile based on normal distribution:

    mean = 0.12764241550612537
    n = 30
    std = 0.02330421008
    [25 percentile, 75 percentile]: [0.1119237258,0.1433611052]

    Reccomended bound: 85.66


In [162]:
from spellchecker import SpellChecker

# ‼️ 🛑 Make sure to change the variable below to the desired accuracy bound 🛑  ‼️
bound = 85.66

#initialize dictionary pairing readability scores and article id's
articles_read_scores = {}

#print(len(articles.index))

# goes through each row (article) in the dataframe:
for index in range(0,1):
    article_index = index

    # defining variables
    article_id  = articles['id'].loc[article_index] 
    article_text = articles['fullText'].loc[article_index]
    article_title = articles['title'].loc[article_index]

    # Assign the full text of this article to a variable called `cleaned_article_text`, with text-matcher's Text function
    cleaned_article_text = Text(article_text, article_title)

    word_list = cleaned_article_text.getTokens()
    
    #word_list = ((" ").join(cleaned_article_text)).split(" ")

    #word_list = random.sample(word_list, int(len(word_list)/float(1)))

    #word_list = word_list[:int(len(word_list)*6/10)]

    # finding the document language
    languages = ['en','fr','es','de']
    abbrev_word_list = word_list[:50]
    incorrect = []
    for lg in languages:
        spell = SpellChecker(language = lg)
        misspelled = spell.unknown(abbrev_word_list)
        incorrect.append(len(misspelled))
    lang = languages[incorrect.index(min(incorrect))]

    # find those words that may be misspelled
    spell = SpellChecker(language = lang)
    misspelled = spell.unknown(word_list)

incorrect_percentage = float(min(incorrect))/len(word_list)



{}


# Run OCR On PDF

This cell converts a pdf to an image and then runs ocr on it.    #output the readability score
    incorrect_percentage = float(len(misspelled))/len(word_list)

    if (1 - incorrect_percentage) < float(bound)/100:
        articles_read_scores[article_id] = [article_index, article_title, incorrect_percentage]

print(articles_read_scores)

In [44]:
from PIL import Image
import pytesseract
import urllib.request
from pdf2image import convert_from_path

pdf = "test_pdf.pdf"
pages = convert_from_path(pdf)

new_ocr_text = ""
for page in pages:
    new_ocr_text += pytesseract.image_to_string(page, lang="eng")

KeyboardInterrupt: 

# Rerun Spellcheck

In [42]:
spell_check_language = 'en'
word_list = new_ocr_text.split(" ")
spell = SpellChecker(language = spell_check_language)

misspelled = spell.unknown(word_list)
incorrect_percentage = float(len(misspelled))/len(word_list)